In [ ]:
!pip install gspread
!pip install oauth2client

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import gspread
from google.colab import auth
import pandas as pd
import math

In [ ]:

# Load the Google Sheet
sheet_url = 'https://docs.google.com/spreadsheets/d/1Yi_xPdR-ioa9HpkYFU7lW4bcarz3OZahH-91xMiXbJI/edit?usp=sharing'
spreadsheet = gc.open_by_url(sheet_url)

In [ ]:

worksheet = gc.open_by_url(sheet_url).sheet1

# Get all records from the sheet
records = worksheet.get_all_records()

# Convert records to DataFrame
ndf = pd.DataFrame(records)
df = ndf.iloc[:, 2:]  # Assuming the first two columns are Timestamp and Name

# Define mapping for converting responses to numeric values
response_mapping = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

# Define function to convert response to numeric value
def convert_response_to_numeric(response):
    return response_mapping.get(response, 0)

# Define function to handle reverse scoring
def handle_reverse_scoring(score, reverse):
    return 6 - score if reverse else score

# Define scales
domain_scales = {
    'Extraversion': ['1', '6', '11R', '16R', '21', '26R', '31R', '36R', '41', '46', '51R', '56'],
    'Agreeableness': ['2', '7', '12R', '17R', '22R', '27', '32', '37R', '42R', '47R', '52', '57'],
    'Conscientiousness': ['3R', '8R', '13', '18', '23R', '28R', '33', '38', '43', '48R', '53', '58R'],
    'Negative Emotionality': ['4R', '9R', '14', '19', '24R', '29R', '34', '39', '44R', '49R', '54', '59'],
    'Open-Mindedness': ['5R', '10', '15', '20', '25R', '30R', '35', '40', '45R', '50R', '55R', '60']
}

facet_scales = {
    'Sociability': ['1', '16R', '31R', '46'],
    'Assertiveness': ['6', '21', '36R', '51R'],
    'Energy Level': ['11R', '26R', '41', '56'],
    'Compassion': ['2', '17R', '32', '47R'],
    'Respectfulness': ['7', '22R', '37R', '52'],
    'Trust': ['12R', '27', '42R', '57'],
    'Organization': ['3R', '18', '33', '48R'],
    'Productiveness': ['8R', '23R', '38', '53'],
    'Responsibility': ['13', '28R', '43', '58R'],
    'Anxiety': ['4R', '19', '34', '49R'],
    'Depression': ['9R', '24R', '39', '54'],
    'Emotional Volatility': ['14', '29R', '44R', '59'],
    'Intellectual Curiosity': ['10', '25R', '40', '55R'],
    'Aesthetic Sensitivity': ['5R', '20', '35', '50R'],
    'Creative Imagination': ['15', '30R', '45R', '60']
}

# Define function to calculate sum for scales
def calculate_scale_sum(row, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        total = 0
        for question in questions:
            question_index = int(question[:-1]) - 1 if question.endswith('R') else int(question) - 1
            reverse = question.endswith('R')
            response = row.iloc[question_index]
            score = convert_response_to_numeric(response)
            score = handle_reverse_scoring(score, reverse)
            total += score
        scale_sum[scale] = total
    return scale_sum

# Define function to map numeric scores to categories
def map_to_category(score):
    if score == 1:
        return "Very Low"
    elif score == 2:
        return "Low"
    elif score == 3:
        return "Medium"
    elif score == 4:
        return "High"
    elif score == 5:
        return "Very High"
    else:
        return "Unknown"
def custom_round(x):
    """Custom rounding function to round halfway values up"""
    integer_part = int(x)
    decimal_part = x - integer_part
    if decimal_part >= 0.5:
        return integer_part + 1
    else:
        return round(x)

# Process each row to calculate domain and facet scale categories
results = []
for index, row in df.iterrows():
    domain_sums = calculate_scale_sum(row, domain_scales)
    domain_avgs = {scale: total / 12 for scale, total in domain_sums.items()}
    domain_categories = {scale: map_to_category(custom_round(avg)) for scale, avg in domain_avgs.items()}

    facet_sums = calculate_scale_sum(row, facet_scales)
    facet_avgs = {scale: total / 4 for scale, total in facet_sums.items()}
    facet_categories = {scale: map_to_category(custom_round(avg)) for scale, avg in facet_avgs.items()}

    result = {**domain_categories, **facet_categories}
    result['Name:'] = ndf.loc[index, 'Name:']
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Reorder columns
columns = list(results_df.columns)
columns.remove('Name:')
columns.insert(0, 'Name:')
results_df = results_df[columns]

# Print results
results_df


,Name:,Extraversion,Agreeableness,Conscientiousness,Negative Emotionality,Open-Mindedness,Sociability,Assertiveness,Energy Level,Compassion,...,Trust,Organization,Productiveness,Responsibility,Anxiety,Depression,Emotional Volatility,Intellectual Curiosity,Aesthetic Sensitivity,Creative Imagination
0,Masum,Medium,High,High,Low,High,Low,Medium,Medium,High,...,High,High,High,High,Medium,Low,Low,High,High,High
1,Reshad,Medium,High,High,Medium,High,Low,Medium,Medium,High,...,High,High,High,High,Medium,Medium,Low,High,High,High
2,,Medium,Medium,Medium,Medium,Medium,High,Medium,Medium,Medium,...,Medium,Medium,High,High,Medium,High,Low,Medium,Medium,Medium


In [ ]:
# Append results_df to the second sheet
worksheet2 = spreadsheet.get_worksheet(1)  # Access the second sheet
worksheet2.clear()  # Clear the sheet first if you want to overwrite the content

# Add the DataFrame to the sheet
worksheet2.update([results_df.columns.values.tolist()] + results_df.values.tolist())

{'spreadsheetId': '1Yi_xPdR-ioa9HpkYFU7lW4bcarz3OZahH-91xMiXbJI',
 'updatedRange': "'Copy of Form responses 1'!A1:U4",
 'updatedRows': 4,
 'updatedColumns': 21,
 'updatedCells': 84}